In [ ]:
import pickle
import os
import pandas as pd
import random
from copy import deepcopy
from collections import defaultdict
import geopandas as gp
from matplotlib import pyplot as plt
from pam.core import Population, Household, Person
from pam.activity import Plan, Activity, Leg
from pam.utils import minutes_to_datetime as mtdt
from pam.variables import END_OF_DAY

from pam.policy import *

In [ ]:
def hhld_apply(household, policy):
    new_hhld = deepcopy(household) 
    policy.apply_to(new_hhld)
    return new_hhld

def print_attributes(hhld, attribs=None):
    if attribs is not None:
        print(f"Household: {hhld}")
        for pid, person in hhld.people.items():
            print(f"Person: {pid}")
            print(f"Attributes:")
            for attrib in attribs:
                print(f"\t{attrib}: {person.attributes[attrib]}")
    else:
        print(f"Household: {hhld}")
        for pid, person in hhld.people.items():
            print(f"Person: {pid}")
            print(f"Attributes:")
            pp.pprint(person.attributes)
            
def instantiate_household_with(persons: list):
    household = Household(1)
    for person in persons:
        household.add(person)
    return household

def Steve():
    Steve = Person('Steve', attributes={'age': 50, 'job': 'work', 'gender': 'male'})
    Steve.add(Activity(1, 'home', 'a', start_time=mtdt(0), end_time=mtdt(5 * 60)))
    Steve.add(Leg(1, 'car', 'a', 'b', start_time=mtdt(5 * 60), end_time=mtdt(6 * 60)))
    Steve.add(Activity(2, 'work', 'b', start_time=mtdt(6 * 60), end_time=mtdt(12 * 60)))
    Steve.add(Leg(2, 'walk', 'b', 'c', start_time=mtdt(12 * 60), end_time=mtdt(12 * 60 + 10)))
    Steve.add(Activity(3, 'leisure', 'c', start_time=mtdt(12 * 60 + 10), end_time=mtdt(13 * 60 - 10)))
    Steve.add(Leg(3, 'walk', 'c', 'b', start_time=mtdt(13 * 60 - 10), end_time=mtdt(13 * 60)))
    Steve.add(Activity(4, 'work', 'b', start_time=mtdt(13 * 60), end_time=mtdt(18 * 60)))
    Steve.add(Leg(4, 'car', 'b', 'a', start_time=mtdt(18 * 60), end_time=mtdt(19 * 60)))
    Steve.add(Activity(5, 'home', 'a', start_time=mtdt(19 * 60), end_time=mtdt(20 * 60)))
    Steve.add(Leg(5, 'car', 'a', 'b', start_time=mtdt(20 * 60), end_time=mtdt(20 * 60 + 20)))
    Steve.add(Activity(6, 'shop_1', 'b', start_time=mtdt(20 * 60 + 20), end_time=mtdt(20 * 60 + 50)))
    Steve.add(Leg(6, 'walk', 'b', 'b', start_time=mtdt(20 * 60 + 50), end_time=mtdt(21 * 60)))
    Steve.add(Activity(7, 'shop_2', 'b', start_time=mtdt(21 * 60), end_time=mtdt(21 * 60 + 50)))
    Steve.add(Leg(7, 'car', 'b', 'a', start_time=mtdt(21 * 60 + 50), end_time=mtdt(22 * 60 + 10)))
    Steve.add(Activity(8, 'home', 'a', start_time=mtdt(22 * 60 + 10), end_time=END_OF_DAY))
    return Steve

def Hilda():
    Hilda = Person('Hilda', attributes={'age': 45, 'job': 'influencer', 'gender': 'female'})
    Hilda.add(Activity(1, 'home', 'a', start_time=mtdt(0), end_time=mtdt(8 * 60)))
    Hilda.add(Leg(1, 'walk', 'a', 'b', start_time=mtdt(8 * 60), end_time=mtdt(8 * 60 + 5)))
    Hilda.add(Activity(2, 'escort', 'b', start_time=mtdt(8 * 60 + 5), end_time=mtdt(8 * 60 + 30)))
    Hilda.add(Leg(1, 'pt', 'a', 'b', start_time=mtdt(8 * 60), end_time=mtdt(8 * 60 + 30)))
    Hilda.add(Activity(2, 'shop', 'b', start_time=mtdt(8 * 60 + 30), end_time=mtdt(14 * 60)))
    Hilda.add(Leg(2, 'pt', 'b', 'c', start_time=mtdt(14 * 60), end_time=mtdt(14 * 60 + 20)))
    Hilda.add(Activity(3, 'leisure', 'c', start_time=mtdt(14 * 60 + 20), end_time=mtdt(16 * 60 - 20)))
    Hilda.add(Leg(3, 'pt', 'c', 'b', start_time=mtdt(16 * 60 - 20), end_time=mtdt(16 * 60)))
    Hilda.add(Activity(4, 'escort', 'b', start_time=mtdt(16 * 60), end_time=mtdt(16 * 60 + 30)))
    Hilda.add(Leg(4, 'walk', 'a', 'b', start_time=mtdt(16 * 60 + 30), end_time=mtdt(17 * 60)))
    Hilda.add(Activity(5, 'home', 'a', start_time=mtdt(17 * 60), end_time=mtdt(20 * 60)))
    Hilda.add(Leg(5, 'car', 'a', 'b', start_time=mtdt(20 * 60), end_time=mtdt(20 * 60 + 20)))
    Hilda.add(Activity(6, 'shop_1', 'b', start_time=mtdt(20 * 60 + 20), end_time=mtdt(20 * 60 + 50)))
    Hilda.add(Leg(6, 'walk', 'b', 'b', start_time=mtdt(20 * 60 + 50), end_time=mtdt(21 * 60)))
    Hilda.add(Activity(7, 'shop_2', 'b', start_time=mtdt(21 * 60), end_time=mtdt(21 * 60 + 50)))
    Hilda.add(Leg(7, 'car', 'b', 'a', start_time=mtdt(21 * 60 + 50), end_time=mtdt(22 * 60 + 10)))
    Hilda.add(Activity(8, 'home', 'a', start_time=mtdt(22 * 60 + 10), end_time=END_OF_DAY))
    return Hilda

def Timmy():
    Timmy = Person('Timmy', attributes={'age': 18, 'job': 'education', 'gender': 'male'})
    Timmy.add(Activity(1, 'home', 'a', start_time=mtdt(0), end_time=mtdt(8 * 60)))
    Timmy.add(Leg(1, 'walk', 'a', 'b', start_time=mtdt(8 * 60), end_time=mtdt(8 * 60 + 5)))
    Timmy.add(Activity(2, 'sport', 'b', start_time=mtdt(8 * 60 + 5), end_time=mtdt(8 * 60 + 55)))
    Timmy.add(Leg(2, 'walk', 'b', 'a', start_time=mtdt(8 * 60 + 55), end_time=mtdt(9 * 60)))
    Timmy.add(Activity(3, 'home', 'a', start_time=mtdt(9 * 60), end_time=mtdt(10 * 60)))
    Timmy.add(Leg(3, 'bike', 'a', 'b', start_time=mtdt(10 * 60), end_time=mtdt(11 * 60)))
    Timmy.add(Activity(4, 'education', 'b', start_time=mtdt(11 * 60), end_time=mtdt(13 * 60)))
    Timmy.add(Leg(4, 'bike', 'b', 'c', start_time=mtdt(13 * 60), end_time=mtdt(13 * 60 + 5)))
    Timmy.add(Activity(5, 'shop', 'c', start_time=mtdt(13 * 60 + 5), end_time=mtdt(13 * 60 + 30)))
    Timmy.add(Leg(5, 'bike', 'c', 'b', start_time=mtdt(13 * 60 + 30), end_time=mtdt(13 * 60 + 35)))
    Timmy.add(Activity(6, 'education', 'b', start_time=mtdt(13 * 60 + 35), end_time=mtdt(15 * 60)))
    Timmy.add(Leg(6, 'bike', 'b', 'd', start_time=mtdt(15 * 60), end_time=mtdt(15 * 60 + 10)))
    Timmy.add(Activity(7, 'leisure', 'd', start_time=mtdt(15 * 60 + 10), end_time=mtdt(18 * 60)))
    Timmy.add(Leg(7, 'bike', 'd', 'a', start_time=mtdt(18 * 60), end_time=mtdt(18 * 60 + 20)))
    Timmy.add(Activity(8, 'home', 'a', start_time=mtdt(18 * 60 + 20), end_time=mtdt(20 * 60)))
    Timmy.add(Leg(8, 'car', 'a', 'b', start_time=mtdt(20 * 60), end_time=mtdt(20 * 60 + 20)))
    Timmy.add(Activity(9, 'shop_1', 'b', start_time=mtdt(20 * 60 + 20), end_time=mtdt(20 * 60 + 50)))
    Timmy.add(Leg(9, 'walk', 'b', 'b', start_time=mtdt(20 * 60 + 50), end_time=mtdt(21 * 60)))
    Timmy.add(Activity(10, 'shop_2', 'b', start_time=mtdt(21 * 60), end_time=mtdt(21 * 60 + 50)))
    Timmy.add(Leg(10, 'car', 'b', 'a', start_time=mtdt(21 * 60 + 50), end_time=mtdt(22 * 60 + 10)))
    Timmy.add(Activity(11, 'home', 'a', start_time=mtdt(22 * 60 + 10), end_time=END_OF_DAY))
    return Timmy

def Bobby():
    Bobby = Person('Bobby', attributes={'age': 6, 'job': 'education', 'gender': 'female'})
    Bobby.add(Activity(1, 'home', 'a', start_time=mtdt(0), end_time=mtdt(8 * 60)))
    Bobby.add(Leg(1, 'walk', 'a', 'b', start_time=mtdt(8 * 60), end_time=mtdt(8 * 60 + 30)))
    Bobby.add(Activity(2, 'education', 'b', start_time=mtdt(8 * 60 + 30), end_time=mtdt(16 * 60)))
    Bobby.add(Leg(2, 'walk', 'b', 'c', start_time=mtdt(16 * 60), end_time=mtdt(16 * 60 + 30)))
    Bobby.add(Activity(3, 'home', 'a', start_time=mtdt(16 * 60 + 30), end_time=mtdt(20 * 60)))
    Bobby.add(Leg(3, 'car', 'a', 'b', start_time=mtdt(20 * 60), end_time=mtdt(20 * 60 + 20)))
    Bobby.add(Activity(4, 'shop_1', 'b', start_time=mtdt(20 * 60 + 20), end_time=mtdt(20 * 60 + 50)))
    Bobby.add(Leg(4, 'walk', 'b', 'b', start_time=mtdt(20 * 60 + 50), end_time=mtdt(21 * 60)))
    Bobby.add(Activity(5, 'shop_2', 'b', start_time=mtdt(21 * 60), end_time=mtdt(21 * 60 + 50)))
    Bobby.add(Leg(5, 'car', 'b', 'a', start_time=mtdt(21 * 60 + 50), end_time=mtdt(22 * 60 + 10)))
    Bobby.add(Activity(6, 'home', 'a', start_time=mtdt(22 * 60 + 10), end_time=END_OF_DAY))
    return Bobby


# Sampling and Policy Levels

In [ ]:
smiths = instantiate_household_with([Steve(), Hilda(), Timmy(), Bobby()])

In [ ]:
smiths.plot()

## Household

$$P_{household} = 1 - (1-P_{person})^n$$

If you have a probability that drives of any one person living in the household.

The probability of the household being chosen is then 
$$1 - (1-P)^n$$
where $P$ is the probability any one person being chosen and $n$ is the number of people in the household; $(1-P)^n$ is the probability of no one being picked.

In [ ]:
hhld_apply(smiths, HouseholdPolicy(RemoveActivity(['education']),1)).plot()

## Person

If you have a probability that drives of any one activity a person does.

The probability of the person being chosen is then 
$$1 - (1-P)^n$$
where $P$ is the probability any one (relevant) activity being chosen and $n$ is the number of (relevant) activities in the person's plan; $(1-P)^n$ is the probability of no relevant activities being picked.

In [ ]:
hhld_apply(smiths, PersonPolicy(RemoveActivity(['shop_1', 'shop_2']),0.5)).plot()

## Activity

If you have a probability that drives of any one activity a person does and you want to affect particular activities with no consequence to the person or household.

Notice that given a probability $P$ for an activity you can affect the whole household. The probability of the household being chosen is then 
$$1 - (1-P)^n$$
where $P$ is the probability any one (relevant) activity being chosen and $n$ is then the number of all (relevant) activities in **all of the persons plans within a household**; $(1-P)^n$ is the probability of no relevant activities being picked for the household.

In [ ]:
hhld_apply(smiths, ActivityPolicy(RemoveActivity(['shop_1', 'shop_2']),0.5)).plot()

The Policies expect certain levels of probability. For example, performing a `HouseholdPolicy` is easy, you can affect a whole household using a probability at any level, `household`, `person` or `activity`. Performing an `ActivityPolicy` however means you need information at the level of the activities you want to affect.

|  | HouseholdProbability | PersonProbability | ActivityProbability |
| --- | --- | --- | --- |
| HouseholdPolicy | Yes | Yes | Yes |
| PersonPolicy | --- | Yes | Yes |
| ActivityPolicy | --- | --- | Yes |

### Joint distributions

You can give the `SamplingProbability` classes custom samplers with joint (or not) distributions. That changes the following table

|  | HouseholdProbability | PersonProbability | ActivityProbability |
| --- | --- | --- | --- |
| HouseholdPolicy | $$p$$ | $$1-(1-p)^{n_p}$$ | $$1-(1-p)^{n_{a,h}}$$ |
| PersonPolicy | $$-$$ | $$p$$ | $$1-(1-p)^{n_{a,p}}$$ |
| ActivityPolicy | $$-$$ | $$-$$ | $$p$$ |

where 

- $n_p$ is the number of people in a household
- $n_{a,p}$ is the number of (relevant) activities in a person's plan
- $n_{a,h}$ is the number of (relevant) activities in a household

to

|  | HouseholdProbability | PersonProbability | ActivityProbability |
| --- | --- | --- | --- |
| HouseholdPolicy | $$p$$ | $$1-\prod_x (1-p_x)^{n_x}$$ | $$1-\prod_{x,a}(1-p_{x,a})^{n_{x,a}}$$  |
| PersonPolicy | $$-$$ | $$p_x$$ | $$1-\prod_a (1-p_a)^{n_a}$$ |
| ActivityPolicy | $$-$$ | $$-$$ | $$p_a, p_{x,a}$$ |

where 

- $p_x$ is the probability for person $x$
- $n_x$ is the number of people with probability $p_x$
- $p_a$ is the probability for activity $a$
- $n_a$ is the number of activities $a$ in a person's plan
- $p_{x,a}$ is the probability for activity $a$ for person $x$
- $n_{x,a}$ is the number of activities $a$ in a person $x$ plan

# Modifiers

Modifier perform the **actions** of removing or moving activities on the chosen **Policy Level** and sampled using the chosen **Sampling Level**.

### `RemoveActivity`
Takes a list of activities to be removed.

In [ ]:
hhld_apply(smiths, HouseholdPolicy(RemoveActivity(['education']),1)).plot()

### `ReduceSharedActivity`
This is a `HouseholdPolicy` level modifier which takes a list of activities, checks the household for those being shared (like going shopping together) and selects a person at random to still perform those activities. Those activities will be deleted for everyone else in the household.

In [ ]:
hhld_apply(smiths, HouseholdPolicy(ReduceSharedActivity(['shop_1']),1)).plot()

### `MoveActivityTourToHomeLocation`
Takes a list of activities and moves those to the home location if they form a **tour**. A tour is a chain of activities sandwiched by two home activities (or boundary of persons plan e.g. if person's activities are: `'work', 'home', 'shop', 'work', 'home'` (in that order), their tours are: `['work'] `and `['shop', work']`).

In [ ]:
hhld_apply(smiths, HouseholdPolicy(MoveActivityTourToHomeLocation(['shop_1', 'shop_2']),1)).plot()

The result of this modifier do not show up on the plot. Below are the details of Hilda's original plan.

In [ ]:
smiths['Hilda'].print()

Here are details of Hilda's plan post application of.`MoveActivityTourToHomeLocation` to `'shop'` activities. Compare the location of Activities 6 and 7 with the ones above. At the moment only the Leg's destination and origin get updated, the mode choice and duration remains unchanged.

In [ ]:
hhld_apply(smiths, HouseholdPolicy(MoveActivityTourToHomeLocation(['shop_1', 'shop_2']),1))['Hilda'].print()

# Attribute Filters
`PersonAttributeFilter` is a conveniece class to apply policies to selected people in the population which satisfy given attributes.

In [ ]:
print_attributes(smiths, ['gender', 'age', 'job'])

In [ ]:
def is_male(attribute_value):
    return attribute_value == 'male'
condition = {'gender': is_male}

hhld_apply(smiths, PersonPolicy(RemoveActivity(['shop_1', 'shop_2']),1,PersonAttributeFilter(condition))).plot()

# Policies

## Person-based Household Quarantine

In [ ]:
smiths.plot()

In [ ]:
policy_household_quarantine_per_person = \
    HouseholdQuarantined(
        PersonProbability(0.01)
)

In [ ]:
hhld_apply(smiths, policy_household_quarantine_per_person).plot()

In [ ]:
hhld_apply(smiths, HouseholdQuarantined(1)).plot()

## Remove Higher Education

Remove all education activity for persons over age of 17

In [ ]:
print_attributes(smiths, ['age', 'job'])

In [ ]:
smiths.plot()

In [ ]:
def over_17(attribute_value):
    return attribute_value > 17

policy_remove_higher_education = \
    PersonPolicy(
        RemoveActivity(['education']),
        PersonProbability(1),
        PersonAttributeFilter({'age': over_17}, how='all')
)

In [ ]:
hhld_apply(smiths, policy_remove_higher_education).plot()

## Remove Education

Probabilistically remove education activities from a person, and escort from people who share the household.
Contraint based on the house not being **care constrainted** and people being able to **work from home**.

### Example of non care constrained household

Let's make the Smiths household a non care constrained household with at least one adult is able to work from home and the household is not care constrained.

In [ ]:
smiths['Steve'].attributes['wfh'] = True
smiths['Steve'].attributes['care_constrained'] = False
smiths['Hilda'].attributes['wfh'] = True
smiths['Hilda'].attributes['care_constrained'] = False
smiths['Timmy'].attributes['wfh'] = False
smiths['Timmy'].attributes['care_constrained'] = False
smiths['Bobby'].attributes['wfh'] = False
smiths['Bobby'].attributes['care_constrained'] = False

In [ ]:
print_attributes(smiths, ['wfh', 'care_constrained'])

In [ ]:
def attribute_False(attribute_value):
    return not bool(attribute_value)

def attribute_True(attribute_value):
    return bool(attribute_value)

conditions = {
    'care_constrained': attribute_False,
    'wfh': attribute_True
}

# notice that the probability of the household (`HouseholdPolicy`) being chosen is driven
# by the presence of education activities (`ActivityProbability(['education'], 0.95)`)
# in the end the policy removes both education and escort (`RemoveActivity(['education', 'escort'])`)
# but the presence of escort doesn't affect the likelihood
policy_remove_any_education = \
    HouseholdPolicy(
        RemoveActivity(['education', 'escort']), 
        ActivityProbability(['education'], 0.95),
        PersonAttributeFilter(conditions)
)

### Non care constrained household is affected

In [ ]:
hhld_apply(smiths, policy_remove_any_education).plot()

### Example of care constrained household

In [ ]:
smiths['Steve'].attributes['wfh'] = False
smiths['Steve'].attributes['care_constrained'] = True
smiths['Hilda'].attributes['wfh'] = False
smiths['Hilda'].attributes['care_constrained'] = True
smiths['Timmy'].attributes['wfh'] = False
smiths['Timmy'].attributes['care_constrained'] = False
smiths['Bobby'].attributes['wfh'] = False
smiths['Bobby'].attributes['care_constrained'] = False

print_attributes(smiths, ['wfh', 'care_constrained'])

### Care constrained household is not affected

In [ ]:
hhld_apply(smiths, policy_remove_any_education).plot()

## Remove Leisure Activities

Remove all leisure activities

In [ ]:
policy_remove_leisure = \
    PersonPolicy(
        RemoveActivity(['leisure']), 
        PersonProbability(1)
)

In [ ]:
hhld_apply(smiths, policy_remove_leisure).plot()

## Move Sport Activities

Probabilistically move Sport activities to home location

In [ ]:
smiths['Timmy'].print()

In [ ]:
policy_move_sport = \
    PersonPolicy(
        MoveActivityTourToHomeLocation(['sport']), 
        PersonProbability(1)
)

In [ ]:
hhld_apply(smiths, policy_move_sport)['Timmy'].print()

## Remove Health Activities

Probabilistically remove **individual** health activities from a person. Make vulnerable people more likely to have their health activities removed.

In [ ]:
def discrete_joint_distribution_sampler(person, mapping, distribution):
    p = distribution
    for key in mapping:
        value = person.attributes.get(key)
        if value is None:
            raise KeyError(f"Cannot find mapping: {key} in sampling features: {features}")
        p = p.get(value)
        if p is None:
            raise KeyError(f"Cannot find feature for {key}: {value} in distribution: {p}")
    return p

In [ ]:
vulnerable_mapping = ['age', 'gender']
vulnerable_distribution = dict(zip(list(range(101)), [{'male': i/100, 'female': i/100, 'other': i/100} for i in range(101)]))

In [ ]:
dict(list(vulnerable_distribution.items())[0:15])

In [ ]:
policy_remove_health = \
    PersonPolicy(
        RemoveActivity(['medical']), 
        [PersonProbability(discrete_joint_distribution_sampler, 
                           {'mapping':vulnerable_mapping, 'distribution':vulnerable_distribution}),
         ActivityProbability(['medical'], 0.5)]
)

In [ ]:
print_attributes(smiths, ['age', 'gender'])

In [ ]:
smiths.plot()

In [ ]:
hhld_apply(smiths, policy_remove_health).plot()

## Unemployment/Furlough

Probabilistically remove all work activities from a person

In [ ]:
policy_unemployment_and_furlough = \
    PersonPolicy(
        RemoveActivity(['work']), 
        PersonProbability(0.1)
)

In [ ]:
hhld_apply(smiths, PersonPolicy(RemoveActivity(['work']), 1)).plot()

## Work from Home

Probabilistically remove all work activities from a person

In [ ]:
smiths['Steve'].attributes['wfh'] = True
smiths['Steve'].attributes['key_worker'] = False
smiths['Hilda'].attributes['wfh'] = False
smiths['Hilda'].attributes['key_worker'] = False
smiths['Timmy'].attributes['wfh'] = False
smiths['Timmy'].attributes['key_worker'] = False
smiths['Bobby'].attributes['wfh'] = False
smiths['Bobby'].attributes['key_worker'] = False

print_attributes(smiths, ['wfh', 'key_worker'])

In [ ]:
conditions = {
    'key_worker': attribute_False,
    'wfh': attribute_True
}

In [ ]:
policy_work_from_home = \
    PersonPolicy(
        RemoveActivity(['work']), 
        PersonProbability(0.5),
        PersonAttributeFilter(conditions)
)

In [ ]:
hhld_apply(smiths, PersonPolicy(RemoveActivity(['work']), 1, 
                                PersonAttributeFilter(conditions))).plot()

In [ ]:
smiths['Steve'].attributes['wfh'] = False
smiths['Steve'].attributes['key_worker'] = True
smiths['Hilda'].attributes['wfh'] = False
smiths['Hilda'].attributes['key_worker'] = False
smiths['Timmy'].attributes['wfh'] = False
smiths['Timmy'].attributes['key_worker'] = False
smiths['Bobby'].attributes['wfh'] = False
smiths['Bobby'].attributes['key_worker'] = False

print_attributes(smiths, ['wfh', 'key_worker'])

In [ ]:
hhld_apply(smiths, PersonPolicy(RemoveActivity(['work']), 1, 
                                               PersonAttributeFilter(conditions))).plot()

## Reduced Work Activity

Probabilistically remove **individual** work activities from a person

In [ ]:
policy_reduced_work_activity = \
    ActivityPolicy(
        RemoveActivity(['work']), 
        ActivityProbability(['work'], 0.2)
)

In [ ]:
hhld_apply(smiths, policy_reduced_work_activity).plot()

## Shopping 


### Reduce Shared Shopping Activities

In [ ]:
policy_reduce_shopping_activities = \
    HouseholdPolicy(
        ReduceSharedActivity(['shop', 'shop_1', 'shop_2']), 
        ActivityProbability(['shop', 'shop_1', 'shop_2'], 1)
)

In [ ]:
smiths_shop = hhld_apply(smiths, policy_reduce_shopping_activities)

In [ ]:
smiths_shop.plot()

In [ ]:
person_with_shopping = [p for p in smiths_shop.people.values() if 'shop_1' in [act.act for act in p.activities]][0].pid

In [ ]:
smiths_shop.people[person_with_shopping].print()

### Move Shopping Activities closer to home

In [ ]:
policy_move_shopping = \
    PersonPolicy(
        MoveActivityTourToHomeLocation(['shop_1', 'shop', 'shop_2']), 
        ActivityProbability(['shop_1', 'shop', 'shop_2'], 1.)
)

In [ ]:
smiths_shop = hhld_apply(smiths_shop, policy_move_shopping)

In [ ]:
smiths_shop.plot()

In [ ]:
smiths.people[person_with_shopping].print()

In [ ]:
smiths_shop.people[person_with_shopping].print()

## All together now!

In [ ]:
population = Population(1)
population.add(smiths)

all_together_pop = apply_policies(
    population, 
    [policy_household_quarantine_per_person, 
     policy_remove_higher_education, 
     policy_remove_any_education, 
     policy_remove_leisure, 
     policy_move_sport,
     policy_remove_health, 
     policy_unemployment_and_furlough, 
     policy_work_from_home, 
     policy_reduced_work_activity, 
     policy_reduce_shopping_activities,
     policy_move_shopping
    ])